<a href="https://colab.research.google.com/github/thomasavare/DNLP-project/blob/master/SPECTER_mag_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook aims at incorporating the MAG classification presented in the [Scidocs](https://github.com/allenai/scidocs) github. This classification consists of 19 different classes.

| int | Class label | int | Class label |
| --- | ----------- | --- | ----------- |
| 0   | Art         | 10	| History     |
| 1	  | Biology     | 11	| Materials science |
| 2	  | Business    | 12	| Mathematics |
| 3	  | Chemistry   | 13	| Medicine    |
| 4   |	Computer science | 14	| Philosophy  |
| 5	  | Economics   | 15	| Physics     |
| 6	  | Engineering | 16	| Political science |
| 7	  | Environmental science | 17 | Psychology  |
| 8	  | Geography   | 18	| Sociology   |
| 9	  | Geology     |

Everything said, let's begin.

First we import the [SPECTER](https://github.com/allenai/specter)embedding model. We are going to use the model accessible through the HuggingFace's transformer librairy.

In [1]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from sentence_transformers import SentenceTransformer, util

# load model and tokenizer
model = SentenceTransformer('allenai-specter')

We are now going to import the data used to train the models in the scidocs benchmarking. This include the meta-data (paper id, title, abstract and citation graph) and also the class associated to these paper.

In [3]:
!pip install awscli
!aws s3 sync --no-sign-request s3://ai2-s2-research-public/specter/scidocs/ data/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd

train_path, test_path = '/content/data/mag/train.csv', '/content/data/mag/test.csv'
train_df, test_df = pd.read_csv(train_path), pd.read_csv(test_path)
train_df

,pid,class_label
0,78f6417635b95decc79e83d42812de50906a562c,10
1,bc636605d012c2a118e8390996843ffe9dc047b0,11
2,a093926d80986aaa93fdc609ce8b125954b87f6b,11
3,84dd81ec2622c433d26f8908d4bc1afd3bff8f6c,18
4,026a5dc5aeddd00b2323b2bedfc91b058c145934,17
...,...,...
17496,f648e027d210e68578d46501f14e581721f449aa,1
17497,78a254f0e8a03762cca5bbc0c7b0f39fc58229ca,2
17498,2eb64dd80227373760ac96fa08a46656e6516cd7,16
17499,2ecca4407aed9ecddac4578cd880e14f82600d0b,1


Unfortunatly, the HuggingFace SPECTER model isn't coinciding with the embeddings provided, so we have to recompute the embeddings. I did it once and dowloaded the data such that I will never have to do it again because it took 3 hours. Here's the code that I used.

----
```{python}
from tqdm import tqdm

for index, row in tqdm(data_df.iterrows(), total=len(data_df.index)):
  data_df['embedding'][row['paper_id']] = model.encode(row['title'] + '[SEP]' + row['title'])
```

100%|██████████| 48473/48473 [3:05:07<00:00, 4.36it/s]

----

The data is currently on my personal drive but I will upload it on my github so it can be easily imported and save 3 hours if someone is using that notebook one day.

In [5]:
embeddings_df = pd.read_json('/content/drive/MyDrive/DNLP-project/embeddings_metadata_mag_mesh.json', orient='index')

In [6]:
embeddings_df

,paper_id,title,abstract,embedding
00014a8515491f0b3fe2a1ff6e0f5305e584dcd9,00014a8515491f0b3fe2a1ff6e0f5305e584dcd9,ON THE CLASSIFICATION OF THE SCIENCES,None,"[-0.7367060184, 0.8094774485, -0.0390381552, -..."
00021eeee2bf4e06fec98941206f97083c38b54d,00021eeee2bf4e06fec98941206f97083c38b54d,Opportunities and challenges for E-Commerce in...,Numerous studies of E-Commerce have emphasized...,"[-1.1841081381, 1.1308963299, 0.6682422161, 0...."
00027baa2a90e1a3d50c1da14882d518de6847f5,00027baa2a90e1a3d50c1da14882d518de6847f5,Interactions between model membranes and ligni...,In order to elucidate the modes of interaction...,"[0.2389703691, 0.7384790778, 1.0119105577, -0...."
00034a5a5bd11b51ec046d31de273946d91fb766,00034a5a5bd11b51ec046d31de273946d91fb766,Perpetual Peace: What Kant Should Have Said,None,"[-0.7743704319, 0.5894224048, -0.3162544966, 0..."
000c8d85037886c86de15290e5a8e9bae7b66103,000c8d85037886c86de15290e5a8e9bae7b66103,Reimagining Greek Tragedy on the American Stage,List of Illustrations Preface Introduction CHA...,"[0.0163563173, 0.50049299, -0.0730320513, 0.64..."
...,...,...,...,...
ffe534bb74efd04b0038510be2a4ed5072430c8b,ffe534bb74efd04b0038510be2a4ed5072430c8b,Comparison of olmesartan combined with a calci...,The cardiovascular effects of combined therapy...,"[-0.5972291231, 0.17011463640000002, 0.6428002..."
ffeaac2b94fc298676e9784ed3bd7a6a7c23b9d1,ffeaac2b94fc298676e9784ed3bd7a6a7c23b9d1,Role of angiotensin II in plasma PAI-1 changes...,To evaluate the relationship between plasma pl...,"[-0.3963649869, 0.5153160095, 0.4946651161, -0..."
ffecb03cbf4dccf9fcdc8b26f40f53d5bc44be66,ffecb03cbf4dccf9fcdc8b26f40f53d5bc44be66,Factors that influence cancer patients' anxiet...,BACKGROUND\nNo study has yet assessed the impa...,"[-0.3526456356, 0.3627114594, -0.8083767891, 0..."
fff238844076ad5643dc2ff53153581bd89441ea,fff238844076ad5643dc2ff53153581bd89441ea,Is adjuvant chemotherapy indicated in stage I ...,OBJECTIVE\nConservative surgery followed by pl...,"[0.5204859972, -0.07988817990000001, -0.430343..."


Now we import the associated MAG classification and merge it with the embedding dataframe.

In [7]:
train_path, test_path = '/content/data/mag/train.csv', '/content/data/mag/test.csv'
train_df, test_df = pd.read_csv(train_path), pd.read_csv(test_path)
train_df.columns

Index(['pid', 'class_label'], dtype='object')

In [8]:
train_merged_df, test_merged_df = pd.merge(embeddings_df, train_df, how='inner', left_on='paper_id', right_on='pid'), pd.merge(embeddings_df, test_df, how='inner', left_on='paper_id', right_on='pid')
train_merged_df, test_merged_df = train_merged_df[['paper_id', 'embedding', 'class_label']], test_merged_df[['paper_id', 'embedding', 'class_label']]

In [9]:
train_merged_df

,paper_id,embedding,class_label
0,00021eeee2bf4e06fec98941206f97083c38b54d,"[-1.1841081381, 1.1308963299, 0.6682422161, 0....",2
1,00027baa2a90e1a3d50c1da14882d518de6847f5,"[0.2389703691, 0.7384790778, 1.0119105577, -0....",3
2,001d134f355306faeb8993fe487a34e0ed0d278b,"[-1.0588871241, 0.0980567485, 0.05070229990000...",10
3,001fb24b565bf8a5faf5551f753a55f3eed6c81c,"[-0.7206891179, 0.6086265445, -0.0693209022, -...",10
4,002087b2c4408d83bc095c835ce70e3974d830c7,"[0.0442002341, 0.0344819799, 0.408043295100000...",9
...,...,...,...
17496,ffeb51c4b44028bc1d5d71851f08d571c3177368,"[-0.3375469446, 0.5189490914, -0.2140062898, -...",12
17497,ffef3035f600d2e8cbb91192128d9315b800f71a,"[-0.33342462780000004, 0.8680065274000001, 0.2...",17
17498,fff1c7f9478f1220c9f9ea341e13651cbfc2996b,"[-1.2471318245, 0.6116715074, 0.36738637090000...",15
17499,fff73361b36bda4423c75f7e9a99e3996947a3c1,"[-1.0980789661, 0.3576785326, -0.2542884648, -...",16


Perfect. Now that we have our train and test data, we can focus on training different models.

Let's put the data in an adequat format.

In [10]:
import numpy as np

x_train, y_train = [], []
x_test, y_test = [], []

i = 0
for _, embedding, class_label in train_merged_df.values:
  x_train.append(embedding)
  y_train.append(class_label)

x_train = np.array(x_train)
y_train = np.array(y_train)

for _, embedding, class_label in test_merged_df.values:
  x_test.append(embedding)
  y_test.append(class_label)

x_test = np.array(x_test)
y_test = np.array(y_test)

First let's try a linear SVC.

In [11]:
from sklearn import svm

classifier1 = svm.LinearSVC(penalty='l2', loss='squared_hinge', random_state=42)

In [12]:
classifier1.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

In [13]:
from sklearn.metrics import f1_score

y_pred = classifier1.predict(x_test)
classifier1.score(x_test, y_test), f1_score(y_test, y_pred, average='macro')

(0.7206078379098907, 0.7105471368841444)

Let's put it to the test with different articles.

In [14]:
def classify_article(classifier, title, abstract, cls=False, probas=False):
  classes = {0: "Art", 1: "Biology", 2: "Business", 3: "Chemistry", 4: "Computer science", 5: "Economics", 6: "Engineering", 7: "Environmental science",\
             8: "Geography", 9: "Geology", 10: "History", 11: "Materials science", 12: "Mathematics", 13: "Medicine", 14: "Philosophy", 15: "Physics",\
             16: "Political science", 17: "Psychology", 18: "Sociology"}

  embed = model.encode([title + '[SEP]' + abstract])

  if cls:
    return np.array(list(zip(classifier.decision_function(embed)[0], classes.values())), dtype=object)
  return classes[classifier.predict(embed)[0]]

Title: SPECTER: Document-level Representation Learning using Citation-informed Transformers

Abstract: Representation learning is a critical ingredient for natural language processing systems. Recent Transformer language models like BERT learn powerful textual representations, but these models are targeted towards token- and sentence-level training objectives and do not leverage information on inter-document relatedness, which limits their document-level representation power. For applications on scientific documents, such as classification and recommendation, the embeddings power strong performance on end tasks. We propose SPECTER, a new method to generate document-level embedding of scientific documents based on pretraining a Transformer language model on a powerful signal of document-level relatedness: the citation graph. Unlike existing pretrained language models, SPECTER can be easily applied to downstream applications without task-specific fine-tuning. Additionally, to encourage further research on document-level models, we introduce SCIDOCS, a new evaluation benchmark consisting of seven document-level tasks ranging from citation prediction, to document classification and recommendation. We show that SPECTER outperforms a variety of competitive baselines on the benchmark.

Class: Computer science (obviously)


In [15]:
res = classify_article(classifier1, "SPECTER: Document-level Representation Learning using Citation-informed Transformers", "Representation learning is a critical ingredient for natural language processing systems. Recent Transformer language models like BERT learn powerful textual representations, but these models are targeted towards token- and sentence-level training objectives and do not leverage information on inter-document relatedness, which limits their document-level representation power. For applications on scientific documents, such as classification and recommendation, the embeddings power strong performance on end tasks. We propose SPECTER, a new method to generate document-level embedding of scientific documents based on pretraining a Transformer language model on a powerful signal of document-level relatedness: the citation graph. Unlike existing pretrained language models, SPECTER can be easily applied to downstream applications without task-specific fine-tuning. Additionally, to encourage further research on document-level models, we introduce SCIDOCS, a new evaluation benchmark consisting of seven document-level tasks ranging from citation prediction, to document classification and recommendation. We show that SPECTER outperforms a variety of competitive baselines on the benchmark.", cls=True)
res[np.argsort(-res[:, 0])]

array([[3.2369296722674905, 'Computer science'],
       [-1.1857062291402207, 'Geography'],
       [-1.716168860218461, 'Political science'],
       [-2.1038272318817426, 'Economics'],
       [-2.2295005016877956, 'Art'],
       [-2.291139931264881, 'Sociology'],
       [-2.4857453916888557, 'Mathematics'],
       [-2.565927137039865, 'History'],
       [-3.7247929432156073, 'Environmental science'],
       [-3.7729087297188744, 'Psychology'],
       [-4.368896558508119, 'Chemistry'],
       [-4.412378605587704, 'Philosophy'],
       [-4.70736910432564, 'Engineering'],
       [-4.943433695584021, 'Business'],
       [-5.4857111370386535, 'Geology'],
       [-5.640873947151579, 'Materials science'],
       [-6.079235584645333, 'Physics'],
       [-6.795215135272074, 'Biology'],
       [-7.547455029785584, 'Medicine']], dtype=object)

Successfuly classifies as computer science.

title: Exploring The Concept Of Cubism Art Essay

abstract: Cubism was one of the most influential art movements of the 20th century. It took place between 1907 and about 1914. The innovators of the Cubist movement were Pablo Picasso (Spanish, 1881 1973) and Georges Braque (French, 1882 1963). Cubism was one of the most significant changes in ideas in the history of art. It allowed for the development of many of the abstract modern art movements in areas such as Futurism and Constructivism.

Class: Art

In [16]:
res = classify_article(classifier1, "Exploring The Concept Of Cubism Art Essay", "Cubism was one of the most influential art movements of the 20th century. It took place between 1907 and about 1914. The innovators of the Cubist movement were Pablo Picasso (Spanish, 1881 1973) and Georges Braque (French, 1882 1963). Cubism was one of the most significant changes in ideas in the history of art. It allowed for the development of many of the abstract modern art movements in areas such as Futurism and Constructivism.", cls=True)
res[np.argsort(-res[:, 0])]

array([[0.6367172470143455, 'Art'],
       [-0.02327048836533857, 'Sociology'],
       [-0.23059300416444772, 'Mathematics'],
       [-0.32591475917339363, 'Economics'],
       [-0.8426298229210223, 'History'],
       [-1.7972712431993418, 'Political science'],
       [-2.0760977984100877, 'Philosophy'],
       [-2.2895093194718465, 'Computer science'],
       [-2.4575364442189374, 'Geography'],
       [-2.5867551257263033, 'Psychology'],
       [-3.1378339909753725, 'Physics'],
       [-4.032798679854295, 'Biology'],
       [-4.411666233356051, 'Engineering'],
       [-5.287606310997519, 'Geology'],
       [-5.557289380822116, 'Materials science'],
       [-5.78371795439588, 'Medicine'],
       [-6.7180000353912765, 'Business'],
       [-8.08379126360974, 'Chemistry'],
       [-8.93821187993181, 'Environmental science']], dtype=object)

Great success.

Let's use gridsearch to find the best set of parameters. The training might take a lot of time because it trains. We're also using f1 as a scoring function since it's the scoring function used in the scidoc benchmark.

The gridsearch is a quite long process when it has a lot of parameters. In this case, we test two different loss and 7 different C, so our gridsearch consist of training and comparing 14 different models to extract the best one.

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

parameters = {'loss': ('hinge', 'squared_hinge'), 'C': np.logspace(-4, 2, 7)}
svc = svm.LinearSVC()
clf1 = GridSearchCV(svc, parameters, scoring=make_scorer(f1_score, average="macro"))

In [18]:
clf1.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warning

GridSearchCV(estimator=LinearSVC(),
             param_grid={'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'loss': ('hinge', 'squared_hinge')},
             scoring=make_scorer(f1_score, average=macro))

In [19]:
params = clf1.get_params()
params

{'cv': None,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__class_weight': None,
 'estimator__dual': True,
 'estimator__fit_intercept': True,
 'estimator__intercept_scaling': 1,
 'estimator__loss': 'squared_hinge',
 'estimator__max_iter': 1000,
 'estimator__multi_class': 'ovr',
 'estimator__penalty': 'l2',
 'estimator__random_state': None,
 'estimator__tol': 0.0001,
 'estimator__verbose': 0,
 'estimator': LinearSVC(),
 'n_jobs': None,
 'param_grid': {'loss': ('hinge', 'squared_hinge'),
  'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': make_scorer(f1_score, average=macro),
 'verbose': 0}

```
{'cv': None,
 'error_score': nan,
 'estimator': LinearSVC(),
 'n_jobs': None,
 'param_grid': {'loss': ('hinge', 'squared_hinge'),
  'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': make_scorer(f1_score, average=macro),
 'verbose': 0}
 ```

In [20]:
y_pred = clf1.predict(x_test)
clf1.score(x_test, y_test), np.mean(y_test == y_pred)

(0.7471634790922824, 0.7592641962143428)

f1 = 0.7471

accuracy = 0.7593

Not as good as in the scidocs benchmarking (f1 = 0.8195). But as we can see there's an error in the convergence and it advises to increase the number of iteration. So let's reuse the parameters of the best current model and increase the number of iterations. Right now, the maximum number of iterations is 1000, so let's set it to 50 000. All the other parameters are actually the default parameters.

In [21]:
clf1_2 = svm.LinearSVC(max_iter=50000)

In [22]:
clf1_2.fit(x_train, y_train)

LinearSVC(max_iter=50000)

In [23]:
y_pred = clf1_2.predict(x_test)
clf1_2.score(x_test, y_test), f1_score(y_test, y_pred, average='macro')

(0.7307384697414023, 0.7209592925178011)

Doesn't improve, tried to do a grid search on the C parameter with max_iter=50 000 but failed everytime and took about 7 hours so I'm not doing it.

Let's save the trained model into the Drive, maybe use it later for a more friendly user interface.

In [24]:
import pickle

filename = '/content/drive/MyDrive/DNLP-project/linearSVC_model_mag.sav'
pickle.dump(clf1, open(filename, 'wb'))

----

Let's try a non linear svm now.

In [25]:
from sklearn.model_selection import GridSearchCV

parameters = {'C': np.logspace(-4, 2, 7)}
svc = svm.SVC()
clf2 = GridSearchCV(svc, parameters, scoring=make_scorer(f1_score, average="macro"))

In [26]:
clf2.fit(x_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
             scoring=make_scorer(f1_score, average=macro))

In [27]:
clf2.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__break_ties': False,
 'estimator__cache_size': 200,
 'estimator__class_weight': None,
 'estimator__coef0': 0.0,
 'estimator__decision_function_shape': 'ovr',
 'estimator__degree': 3,
 'estimator__gamma': 'scale',
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__probability': False,
 'estimator__random_state': None,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVC(),
 'n_jobs': None,
 'param_grid': {'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': make_scorer(f1_score, average=macro),
 'verbose': 0}

```
{'cv': None,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__break_ties': False,
 'estimator__cache_size': 200,
 'estimator__class_weight': None,
 'estimator__coef0': 0.0,
 'estimator__decision_function_shape': 'ovr',
 'estimator__degree': 3,
 'estimator__gamma': 'scale',
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__probability': False,
 'estimator__random_state': None,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVC(),
 'n_jobs': None,
 'param_grid': {'C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': make_scorer(f1_score, average=macro),
 'verbose': 0}
```


In [28]:
y_pred = clf2.predict(x_test)
clf2.score(x_test, y_test), np.mean(y_test == y_pred)

(0.7432253137955992, 0.7525993068515062)

f1 = 0.743

accuracy = 0.752

no improvment, equivalent results.

In [29]:
filename = '/content/drive/MyDrive/DNLP-project/SVC_model_mag.sav'
pickle.dump(clf1, open(filename, 'wb'))

----

Let's try naive bayes

In [30]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

In [31]:
gnb.fit(x_train, y_train)

GaussianNB()

In [32]:
y_pred = gnb.predict(x_test)
gnb.score(x_test, y_test), f1_score(y_test, y_pred, average="macro")

(0.7094108237803253, 0.6978830891292475)

Doesn't work well

In [33]:
from sklearn.neighbors import KNeighborsClassifier

knb = KNeighborsClassifier()

In [34]:
knb.fit(x_train, y_train)

KNeighborsClassifier()

In [35]:
y_pred = knb.predict(x_test)
knb.score(x_test, y_test), f1_score(y_test, y_pred, average="macro")

(0.7014129565449213, 0.6925303177923376)

Works poorly, still save it.

In [36]:
filename = '/content/drive/MyDrive/DNLP-project/Knn_model_mag.sav'
pickle.dump(clf1, open(filename, 'wb'))

----

Let's try SGD classifier with gridsearch for the parameters.

In [37]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier('squared_error')

In [38]:
sgd.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDClassifier(loss='squared_error')

In [39]:
y_pred = sgd.predict(x_test)
sgd.score(x_test, y_test), f1_score(y_test, y_pred, average="macro")

(0.053585710477206075, 0.01537701144338236)

f1 = 0.0154

accuracy = 0.0536

That's hilarious, it's the most inaccurate classification model I've ever seen. The accuracy is even lower than 1/19 = 0.05263 which is the theoritical expected value if we have a random uniform classifier.

In [40]:
sgd = SGDClassifier('squared_epsilon_insensitive')

In [41]:
sgd.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDClassifier(loss='squared_epsilon_insensitive')

In [42]:
y_pred = sgd.predict(x_test)
sgd.score(x_test, y_test), f1_score(y_test, y_pred, average="macro")

(0.04585443881631565, 0.018222376229698065)

f1 = 0.0182

accuracy = 0.0458

That's even worse, the thing is that it took quite some time eventhough it's kind of funny

In [43]:
parameters = {'loss': ('hinge', 'modified_huber', 'squared_hinge')}
sgd = SGDClassifier()
clf3 = GridSearchCV(sgd, parameters, scoring=make_scorer(f1_score, average="macro"))

In [44]:
clf3.fit(x_train, y_train)

GridSearchCV(estimator=SGDClassifier(),
             param_grid={'loss': ('hinge', 'modified_huber', 'squared_hinge')},
             scoring=make_scorer(f1_score, average=macro))

In [45]:
clf3.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__alpha': 0.0001,
 'estimator__average': False,
 'estimator__class_weight': None,
 'estimator__early_stopping': False,
 'estimator__epsilon': 0.1,
 'estimator__eta0': 0.0,
 'estimator__fit_intercept': True,
 'estimator__l1_ratio': 0.15,
 'estimator__learning_rate': 'optimal',
 'estimator__loss': 'hinge',
 'estimator__max_iter': 1000,
 'estimator__n_iter_no_change': 5,
 'estimator__n_jobs': None,
 'estimator__penalty': 'l2',
 'estimator__power_t': 0.5,
 'estimator__random_state': None,
 'estimator__shuffle': True,
 'estimator__tol': 0.001,
 'estimator__validation_fraction': 0.1,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': SGDClassifier(),
 'n_jobs': None,
 'param_grid': {'loss': ('hinge', 'modified_huber', 'squared_hinge')},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': make_scorer(f1_score, average=macro),
 'verbose': 0}

In [46]:
y_pred = clf3.predict(x_test)
clf3.score(x_test, y_test), np.mean(y_test == y_pred)

(0.7124041360337442, 0.7243401759530792)

f1 = 0.7124

accuracy = 0.7243

In [47]:
filename = '/content/drive/MyDrive/DNLP-project/SGD_model_mag.sav'
pickle.dump(clf1, open(filename, 'wb'))

----

Let's try a CNN now. first we'll try a very simple CNN with only 2 dense layers.

In [48]:
import tensorflow as tf
y_train_cnn = tf.keras.utils.to_categorical(y_train)
x_train_cnn = np.expand_dims(x_train, -1)

In [49]:
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(x_train.shape[1], 1,), dtype="float")


# We add a vanilla hidden layer:
x = layers.Flatten()(inputs)
x = layers.Dense(x_train.shape[0], activation="relu")(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(19, activation="sigmoid", name="predictions")(x)

Cnn1 = tf.keras.Model(inputs, predictions)

In [50]:
Cnn1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 768, 1)]          0         
                                                                 
 flatten (Flatten)           (None, 768)               0         
                                                                 
 dense (Dense)               (None, 17501)             13458269  
                                                                 
 predictions (Dense)         (None, 19)                332538    
                                                                 
Total params: 13,790,807
Trainable params: 13,790,807
Non-trainable params: 0
_________________________________________________________________


In [51]:
Cnn1.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [52]:
epochs = 10

# Fit the model using the train and test datasets.
Cnn1.fit(x_train_cnn, y_train_cnn, epochs=epochs)

Epoch 1/10
547/547 [==============================] - 52s 93ms/step - loss: 1.0089 - accuracy: 0.6823
Epoch 2/10
547/547 [==============================] - 46s 84ms/step - loss: 0.8127 - accuracy: 0.7286
Epoch 3/10
547/547 [==============================] - 46s 85ms/step - loss: 0.7326 - accuracy: 0.7540
Epoch 4/10
547/547 [==============================] - 46s 85ms/step - loss: 0.6721 - accuracy: 0.7705
Epoch 5/10
547/547 [==============================] - 47s 86ms/step - loss: 0.6137 - accuracy: 0.7866
Epoch 6/10
547/547 [==============================] - 46s 83ms/step - loss: 0.5542 - accuracy: 0.8057
Epoch 7/10
547/547 [==============================] - 46s 84ms/step - loss: 0.4991 - accuracy: 0.8220
Epoch 8/10
547/547 [==============================] - 46s 84ms/step - loss: 0.4348 - accuracy: 0.8454
Epoch 9/10
547/547 [==============================] - 46s 84ms/step - loss: 0.3800 - accuracy: 0.8653
Epoch 10/10
547/547 [==============================] - 47s 85ms/step - loss: 0.330

In [53]:
x_test_cnn = np.expand_dims(x_test, -1)
y_pred = np.argmax(Cnn1.predict(x_test_cnn), axis=1)
np.mean(y_test == y_pred), f1_score(y_test, y_pred, average="macro")

118/118 [==============================] - 3s 29ms/step


(0.7366035723806985, 0.727631239459869)

f1 = 0.7205

accuracy = 0.7323


No big improvment compared to the SVC classification. Let's expand our network. This model is inspired by the "Alexnet" network and was initially designed for 1000 class classification and won the ImageNet Large Scale Visual Recognition Challenge in 2012.

In [54]:
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(x_train.shape[1], 1,), dtype="float")

x = layers.Conv1D(96, 11, 4)(inputs)
x = layers.MaxPool1D(3, 2)(x)
x = layers.Conv1D(256, 5)(x)
x = layers.MaxPool1D(3, 2)(x)
x = layers.Conv1D(384, 3)(x)
x = layers.Conv1D(384, 3)(x)
x = layers.MaxPool1D(3, 2)(x)

x = layers.Flatten()(x)
x = layers.Dense(2048, activation="relu")(x)
x = layers.Dense(2048, activation="relu")(x)


# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(19, activation="sigmoid", name="predictions")(x)

Cnn2 = tf.keras.Model(inputs, predictions)

In [55]:
Cnn2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 768, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 190, 96)           1152      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 94, 96)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 90, 256)           123136    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 44, 256)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 42, 384)           2952

In [56]:
Cnn2.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [59]:
epochs = 10

# Fit the model using the train and test datasets.
Cnn2.fit(x_train_cnn, y_train_cnn, epochs=epochs)

Epoch 1/10
547/547 [==============================] - 243s 444ms/step - loss: 0.5826 - accuracy: 0.8043
Epoch 2/10
547/547 [==============================] - 230s 420ms/step - loss: 0.5432 - accuracy: 0.8201
Epoch 3/10
547/547 [==============================] - 215s 394ms/step - loss: 0.5450 - accuracy: 0.8214
Epoch 4/10
547/547 [==============================] - 216s 395ms/step - loss: 0.4631 - accuracy: 0.8429
Epoch 5/10
547/547 [==============================] - 215s 392ms/step - loss: 0.4940 - accuracy: 0.8387
Epoch 6/10
547/547 [==============================] - 215s 393ms/step - loss: 0.4738 - accuracy: 0.8487
Epoch 7/10
547/547 [==============================] - 215s 394ms/step - loss: 0.4677 - accuracy: 0.8548
Epoch 8/10
547/547 [==============================] - 215s 394ms/step - loss: 0.4228 - accuracy: 0.8643
Epoch 9/10
547/547 [==============================] - 215s 394ms/step - loss: 0.4592 - accuracy: 0.8618
Epoch 10/10
547/547 [==============================] - 217s 398m

In [64]:
x_test_cnn = np.expand_dims(x_test, -1)
y_pred = np.argmax(Cnn2.predict(x_test_cnn), axis=1)
np.mean(y_test == y_pred), f1_score(y_test, y_pred, average="macro")

118/118 [==============================] - 12s 104ms/step


(0.6808850973073847, 0.6761025784765151)

f1 = 0.6825

accuracy = 0.7235 

what a disappointment.

In [1]:
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(x_train.shape[1], 1,), dtype="float")

x = layers.Conv1D(256, 5)(inputs)
x = layers.MaxPool1D(3, 2)(x)

# We add a vanilla hidden layer:
x = layers.Flatten()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(x_train.shape[0], activation="relu")(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(19, activation="sigmoid", name="predictions")(x)

Cnn3 = tf.keras.Model(inputs, predictions)

NameError: ignored

In [ ]:
Cnn3.summary()

In [62]:
Cnn3.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [63]:
epochs = 10

# Fit the model using the train and test datasets.
Cnn3.fit(x_train_cnn, y_train_cnn, epochs=epochs)

Epoch 1/10
 30/547 [>.............................] - ETA: 25:45 - loss: 4.2372 - accuracy: 0.4563

KeyboardInterrupt: ignored

In [ ]:
x_test_cnn = np.expand_dims(x_test, -1)
y_pred = np.argmax(Cnn3.predict(x_test_cnn), axis=1)
np.mean(y_test == y_pred), f1_score(y_test, y_pred, average="macro")